# The Last Samurai's Dinner

## Selecting Data

Let's read the CSV files first.

In [ ]:
#Nec. 1

import numpy as np
import pandas as pd
import random
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


ainf = pd.read_csv('air_store_info.csv')
hinf = pd.read_csv('hpg_store_info.csv')
idrel = pd.read_csv('store_id_relation.csv')

ares = pd.read_csv('air_reserve.csv')
avis = pd.read_csv('air_visit_data.csv')
hres = pd.read_csv('hpg_reserve.csv')
date_info = pd.read_csv('date_info.csv')
sub = pd.read_csv('sample_submission.csv')
sample_submission = sub.copy()

idrel.set_index('hpg_store_id', 'air_store_id')
di = idrel.set_index('hpg_store_id').T.to_dict('list')
dictionary = {}
for key in di:
    dictionary[key]=di[key][0]

hinf["hpg_store_id"].replace(dictionary, inplace=True)
hres["hpg_store_id"].replace(dictionary, inplace=True)
ainf.rename(columns={'air_store_id': 'store_id','air_area_name': 'area_name', 'air_genre_name': 'genre_name'}, inplace=True)
hinf.rename(columns={'hpg_store_id': 'store_id','hpg_area_name': 'area_name', 'hpg_genre_name': 'genre_name'}, inplace=True)
hres.rename(columns={'hpg_store_id': 'store_id'}, inplace=True)
ares.rename(columns={'air_store_id': 'store_id'}, inplace=True)
avis.rename(columns={'air_store_id': 'store_id', 'visit_date': 'vis_date'}, inplace=True)

stsinf = pd.concat([ainf, hinf])
res = pd.concat([ares, hres])
stsinf = stsinf.drop_duplicates(subset=['store_id'], keep='first')
res = res.drop_duplicates(subset=['store_id'], keep='first')
stsinf = stsinf.set_index([list(range(len(stsinf)))])
res = res.set_index([list(range(len(res)))])

stsinf['prefecture'] = stsinf['area_name'].apply(lambda x: x.split('-')[0])
stsinf['prefecture'] = stsinf['prefecture'].apply(lambda x: x.split(' ')[0])
stsinf['prefecture'].replace('None', np.nan, inplace=True)
cleanup_nums = {"prefecture":     {"Tōkyō": 1, "Ōsaka": 2, "Osaka": 2, "Fukuoka": 3, "Hyōgo": 4, "Hokkaidō": 5, "Hiroshima": 6, "Shizuoka": 7, "Niigata": 8, "Miyagi": 9, "Kanagawa": 10, "Saitama": 11},
                "genre_name": {"Japanese style": 1, "Japanese food in general": 1, "Japanese food": 1, "Japanese cuisine/Kaiseki": 1,
                                  "Shabu-shabu/Sukiyaki": 1, "Okonomiyaki/Monja/Teppanyaki": 1, "Udon/Soba":1, "Izakaya": 1,
                               "Seafood": 1, "Sushi": 1, "Grilled meat": 1,
                              "Cafe/Sweets": 2, "Sweets": 2, "Cafe": 2,
                              "Dining bar": 3, "Bar/Cocktail": 3, "Karaoke": 3, "Party": 3, "Amusement bar": 3, "Karaoke/Party": 3,
                              "International cuisine": 4, "Italian": 4, "Spain Bar/Italian Bar": 4, "Italian/French": 4, "French": 4, "Bistro": 4,
                               "Spain/Mediterranean cuisine": 4, "Steak/Hamburger/Curry": 4, "Western food": 4, "Pasta/Pizza": 4,
                              "Chinese general": 5, "Korean cuisine": 5, "Yakiniku/Korean food": 5, "Thai/Vietnamese food": 5,
                               "Cantonese food": 5, "Sichuan food": 5, "Asian": 5,
                              "Dim Sum/Dumplings": 5, "Shanghai food": 5, "Taiwanese/Hong Kong cuisine": 5,
                              "Other": 6,
                               "Creation": 7, "Creative Japanese food": 7, "Creative cuisine": 7},
               "day_of_week": {"Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5, "Saturday": 6, "Sunday": 7}}
stsinf.replace(cleanup_nums, inplace=True)
stsinf = stsinf.drop('area_name', 1)
stsinf['prefecture'] = stsinf['prefecture'].fillna(random.choice(stsinf['prefecture'].values.tolist()))

In [ ]:
stsinf

In [ ]:
while stsinf.prefecture.isnull().sum() > 0:
    index = stsinf['prefecture'].index[stsinf['prefecture'].apply(np.isnan)]
    stsinf.loc[np.sqrt((stsinf.longitude.sub(float(stsinf.loc[stsinf.prefecture.isnull(),'longitude'][:1]))).pow(2)
                                         +(stsinf.latitude.sub(float(stsinf.loc[stsinf.prefecture.isnull(),'latitude'][:1]))).pow(2)+1e-10*stsinf.prefecture)
                                       .idxmin()].prefecture

In [ ]:
stsinf.info()

In [2]:
#Nec. 2


stsinf = stsinf.drop('longitude', 1)
stsinf = stsinf.drop('latitude', 1)

def hr_func(ts):
    return ts.hour


res['visit_datetime'] = res['visit_datetime'].apply(lambda x : pd.to_datetime(str(x)))
res['vis_date'] = res['visit_datetime'].dt.date
res['vis_hour'] = res['visit_datetime'].dt.time

res['reserve_datetime'] = res['reserve_datetime'].apply(lambda x : pd.to_datetime(str(x)))
res['res_date'] = res['reserve_datetime'].dt.date
res['res_hour'] = res['reserve_datetime'].dt.time

avis['vis_date'] = avis['vis_date'].apply(lambda x : pd.to_datetime(str(x)))
avis['vis_date'] = avis['vis_date'].dt.date
res['vis_h'] = res['vis_hour'].apply(hr_func)
res['res_h'] = res['res_hour'].apply(hr_func)
res.drop(['visit_datetime', 'reserve_datetime', 'vis_hour', 'res_hour'], axis=1, inplace=True)
date_info['vis_date'] = date_info['calendar_date'].apply(lambda x : pd.to_datetime(str(x)))
date_info.replace(cleanup_nums, inplace=True)
date_info['vis_date'] = date_info['vis_date'].dt.date
date_info = date_info.drop('calendar_date', 1)
reserve = res.merge(avis, on=['store_id', 'vis_date'], how='outer')
reserve = reserve.merge(date_info, on=['vis_date'], how='outer')
df = reserve.merge(stsinf, on=['store_id'], how='outer')
df.visitors.fillna(value=df['reserve_visitors'].groupby([df['store_id'],df['vis_date']]).transform('sum'), inplace=True)



In [ ]:
df = df[np.isfinite(df['visitors'])]

In [ ]:
#Nec. 2

df["genre_name"].fillna(random.choice(df[df['genre_name'] != np.nan]["genre_name"]).astype(float), inplace =True)
df["prefecture"].fillna(random.choice(df[df['prefecture'] != np.nan]["prefecture"]), inplace =True)
df = df.dropna(subset = ['store_id'])

sub['store_id'] = sub['id'].apply(lambda x: x.split('_')[0]+'_'+x.split('_')[1])
sub['vis_date'] = sub['id'].apply(lambda x: x.split('_')[2])
sub['vis_date'] = sub['vis_date'].apply(lambda x : pd.to_datetime(str(x)))
sub['vis_date'] = sub['vis_date'].dt.date
sub = sub.merge(stsinf, on=['store_id'], how='inner')
sub = sub[['store_id','vis_date','genre_name','prefecture']]
sub = sub.merge(date_info, on='vis_date', how='inner')

In [ ]:
# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# Configure visualisations
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6

def plot_correlation_map( df ):
    corr = df.corr()
    _ , ax = plt.subplots( figsize =( 12 , 10 ) )
    cmap = sns.diverging_palette( 220 , 10 , as_cmap = True )
    _ = sns.heatmap(
        corr, 
        cmap = cmap,
        square=True, 
        cbar_kws={ 'shrink' : .9 }, 
        ax=ax, 
        annot = True, 
        annot_kws = { 'fontsize' : 12 }
    )

plot_correlation_map( df )

In [ ]:
# Plot visitors number by genre
def plot_categories( df , cat , target , **kwargs ):
    row = kwargs.get( 'row' , None )
    col = kwargs.get( 'col' , None )
    facet = sns.FacetGrid( df , row = row , col = col )
    facet.map( sns.barplot , cat , target )
    facet.add_legend()

plot_categories( df , cat = 'genre_name' , target = 'visitors' )

In [ ]:
#Nec. 4


# Create all datasets that are necessary to train, validate and test models
# Modelling Algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

# Modelling Helpers
from sklearn.preprocessing import Imputer , Normalizer , scale
from sklearn.cross_validation import train_test_split , StratifiedKFold
from sklearn.feature_selection import RFECV

train_valid_X = df[['vis_date','genre_name', 'prefecture', 'day_of_week', 'holiday_flg']]

from datetime import datetime
start_date = train_valid_X['vis_date'].min()
train_valid_X['vis_date'] = train_valid_X['vis_date'] - start_date
train_valid_X['vis_date'] = train_valid_X['vis_date'].dt.days

train_valid_y = df['visitors']
test_X = sub
test_X = test_X[['vis_date','genre_name', 'prefecture', 'day_of_week', 'holiday_flg']]
test_X['vis_date'] = test_X['vis_date'] - start_date
test_X['vis_date'] = test_X['vis_date'].dt.days

train_X , valid_X , train_y , valid_y = train_test_split( train_valid_X , train_valid_y , train_size = .7 )

print (train_X.shape , valid_X.shape , train_y.shape , valid_y.shape , test_X.shape)

In [ ]:
test_X

In [ ]:
def plot_model_var_imp( model , X , y ):
    imp = pd.DataFrame( 
        model.feature_importances_  , 
        columns = [ 'Importance' ] , 
        index = X.columns 
    )
    imp = imp.sort_values( [ 'Importance' ] , ascending = True )
    imp[ : 10 ].plot( kind = 'barh' )
    print (model.score( X , y ))

def plot_variable_importance( X , y ):
    tree = DecisionTreeClassifier( random_state = 99 )
    tree.fit( X , y )
    plot_model_var_imp( tree , X , y )
    
plot_variable_importance(train_X, train_y)

In [ ]:
model = LogisticRegression()

model.fit( train_X , train_y )

# Score the model
print (model.score( train_X , train_y ) , model.score( valid_X , valid_y ))

In [ ]:
test_Y = model.predict( test_X )
vis_id = sample_submission.id
test = pd.DataFrame( { 'id': vis_id , 'visitors': test_Y } )
test.shape
test.head()
test.to_csv( 'Recruit_Sub.csv' , index = False )